# Data visualization

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('../dwh.cfg'))

DWH_ENDPOINT           = config.get("CLUSTER", "HOST")
DWH_DB                 = config.get("CLUSTER", "DB_NAME")
DWH_DB_USER            = config.get("CLUSTER", "DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER", "DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER", "DB_PORT")

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

In [ ]:
from plotly.offline import iplot
import plotly.graph_objs as go

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

## Play count, per user plan, per day

In [ ]:
%%time
%%sql pd << 
SELECT 
    to_char(sp.start_time, 'YYYY-MM-DD') as date, 
    sp.level, 
    count(*)
FROM songplays sp
GROUP BY 1, 2
ORDER BY 1

In [ ]:
enable_plotly_in_cell()

pd = pd.DataFrame()

free_dataset = pd[pd.level == 'free']
free_bar = go.Bar(
    x=list(free_dataset['date']),
    y=list(free_dataset['count']),
    name='Free',
    marker=dict(
        color='#d62728'
    )
)

paid_dataset = pd[pd.level == 'paid']
paid_bar = go.Bar(
    x=paid_dataset['date'],
    y=paid_dataset['count'],
    name='Paid',
    marker=dict(
        color='#1f77b4'
    )
)

data = [free_bar, paid_bar]
layout = go.Layout(
    barmode='stack',
    title='Play count, per user plan, per day'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

## Play count, per user gender, per day

In [ ]:
%%time
%%sql pd << 
SELECT 
    to_char(sp.start_time, 'YYYY-MM-DD') as date, 
    u.gender, 
    count(*)
FROM songplays sp
LEFT JOIN users u
    ON u.user_id = sp.user_id
GROUP BY 1, 2
ORDER BY 1

In [ ]:
enable_plotly_in_cell()

pd = pd.DataFrame()

male_dataset = pd[pd.gender == 'M']
male_bar = go.Bar(
    x=list(male_dataset['date']),
    y=list(male_dataset['count']),
    name='Male',
    marker=dict(
        color='#1f77b4'
    )
)

female_dataset = pd[pd.gender == 'F']
female_bar = go.Bar(
    x=female_dataset['date'],
    y=female_dataset['count'],
    name='Female',
    marker=dict(
        color='#d62728'
    )
)

data = [female_bar, male_bar]
layout = go.Layout(
    barmode='stack',
    title='Play count, per user gender, per day'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

## Play count, per artists identified, per day

In [ ]:
%%time
%%sql pd << 
SELECT 
    to_char(sp.start_time, 'YYYY-MM-DD') as date, 
    (sp.artist_id IS NOT NULL) as found_artist, 
    count(*)
FROM songplays sp
GROUP BY 1, 2
ORDER BY 1

In [ ]:
enable_plotly_in_cell()

pd = pd.DataFrame()

found_dataset = pd[pd.found_artist == True]
found_bar = go.Bar(
    x=list(found_dataset['date']),
    y=list(found_dataset['count']),
    name='Artist found',
    marker=dict(
        color='#1f77b4'
    )
)

missing_dataset = pd[pd.found_artist == False]
missing_bar = go.Bar(
    x=missing_dataset['date'],
    y=missing_dataset['count'],
    name='Artist not found',
    marker=dict(
        color='#d62728'
    )
)

data = [found_bar, missing_bar]
layout = go.Layout(
    barmode='stack',
    title='Play count, per artists identified, per day'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

## Play count, per songs identified, per day

In [ ]:
%%time
%%sql pd << 
SELECT 
    to_char(sp.start_time, 'YYYY-MM-DD') as date, 
    (sp.song_id IS NOT NULL) as found_song, 
    count(*)
FROM songplays sp
GROUP BY 1, 2
ORDER BY 1

In [ ]:
enable_plotly_in_cell()

pd = pd.DataFrame()

found_dataset = pd[pd.found_song == True]
found_bar = go.Bar(
    x=list(found_dataset['date']),
    y=list(found_dataset['count']),
    name='Song found',
    marker=dict(
        color='#1f77b4'
    )
)

missing_dataset = pd[pd.found_song == False]
missing_bar = go.Bar(
    x=missing_dataset['date'],
    y=missing_dataset['count'],
    name='Song not found',
    marker=dict(
        color='#d62728'
    )
)

data = [found_bar, missing_bar]
layout = go.Layout(
    barmode='stack',
    title='Play count, per songs identified, per day'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')